# MLflow - toy example

In [1]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import requests
from datetime import datetime

In [2]:
alpha = 1.0
X = [[0, 0], [1, 1], [2, 2]]
y = [0, 1, 2]

In [3]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("linear_regression_experiment")

2025/06/12 13:24:30 INFO mlflow.tracking.fluent: Experiment with name 'linear_regression_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/adrian/projects/2025L-PJATK-SUML/07-ml-tools/mlflow/mlruns/1', creation_time=1749727470732, experiment_id='1', last_update_time=1749727470732, lifecycle_stage='active', name='linear_regression_experiment', tags={}>

### Train model

In [4]:
with mlflow.start_run() as run:
    # Log parameters
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("model_type", "ElasticNet")
    mlflow.log_param("data_source", "synthetic_data")
    
    # Train model
    reg = ElasticNet(alpha=alpha)
    reg.fit(X, y)
    
    # calculate mse and r2 score
    y_pred = reg.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2_score", r2)
    
    # Log model
    mlflow.sklearn.log_model(reg, "model")
    
    run_id = run.info.run_id

print(f"Run ID: {run_id}")

2025/06/12 13:24:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/12 13:24:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run luminous-doe-45 at: http://localhost:5000/#/experiments/1/runs/75440d5793c9468ab0dd141bd7ec9220
🧪 View experiment at: http://localhost:5000/#/experiments/1
Run ID: 75440d5793c9468ab0dd141bd7ec9220


### Register the model

In [5]:
mlflow.register_model(
    f"runs:/{run_id}/model",
    "ElasticNetModel"
)

Successfully registered model 'ElasticNetModel'.
2025/06/12 13:24:34 WARNING mlflow.tracking._model_registry.fluent: Run with id 75440d5793c9468ab0dd141bd7ec9220 has no artifacts at artifact path 'model', registering model based on models:/m-4c3319e4ff534f99b425829ad8545e93 instead
2025/06/12 13:24:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


<ModelVersion: aliases=[], creation_timestamp=1749727474914, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1749727474914, metrics=None, model_id=None, name='ElasticNetModel', params=None, run_id='75440d5793c9468ab0dd141bd7ec9220', run_link='', source='models:/m-4c3319e4ff534f99b425829ad8545e93', status='READY', status_message=None, tags={}, user_id='', version='1'>

### Serve the model

In [6]:
# run in bash
# MLFLOW_TRACKING_URI=http://localhost:5000 mlflow models serve -m models:/ElasticNetModel/1 --port 1234 --no-conda

### Call the model via REST API

In [8]:
data = {
    "instances": [[3, 3], [10, 10]]
}
res = requests.post("http://localhost:1234/invocations", json=data)
print(res.json())

{'predictions': [1.363638242683178, 2.6363720920743003]}
